In [9]:
import pandas as pd
import pymysql

### 빅카인즈에서 제공하는 키워드를 포함하는 뉴스기사수 dataframe으로 만들기
키워드: 침체, 금융위기, 불황, 저성장, 외환위기, 금리인하, 디플레이션, 불경기

In [14]:
news_df = pd.read_json('./data/bigkinds.json')
news_df

,date,침체,금융위기,불황,저성장,외환위기,금리인하,디플레이션,불경기
0,20150907,737,399,273,256,135,80,48,34
1,20150914,701,513,274,232,226,100,39,28
2,20150921,575,267,231,241,111,42,35,24
3,20150928,480,248,173,126,79,31,49,25
4,20151005,576,262,204,216,109,46,39,20
...,...,...,...,...,...,...,...,...,...
264,20200928,489,90,114,27,78,8,4,15
265,20201005,816,183,177,47,78,10,6,23
266,20201012,857,199,176,55,91,21,20,10
267,20201019,911,155,143,64,129,9,6,16


In [34]:
def getDate(df_row):
    date = str(df_row)

    year = date[0:4]
    month = date[4:6]
    day = date[6:9]
    return int(year), int(month), int(day)

### DB연동

In [28]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE], cursorclass = pymysql.cursors.DictCursor)
cur = conn.cursor()

cur.execute("show databases")
print(cur.fetchall())
cur.execute("use CLI")

[{'Database': 'information_schema'}, {'Database': 'CLI'}, {'Database': 'MusicBank'}, {'Database': 'mysql'}, {'Database': 'mytestdb'}, {'Database': 'performance_schema'}, {'Database': 'sys'}]


0

### DB table 생성

keyword1: 침체, keyword2:금융위기, keyword3: 불황, keyword4: 저성장, keyword5: 외환위기

In [29]:
"""
news counting 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS news_counting2(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    day bigint(2) NOT NULL, 
    keyword1 bigint(100),
    keyword2 bigint(100),
    keyword3 bigint(100),
    keyword4 bigint(100),
    keyword5 bigint(100),
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)
# charset=utf8mb4; 유니코드 스트링 저장

0

In [30]:
cur.execute("desc news_counting2")
table = cur.fetchall()
pd.DataFrame(list(table))

,Field,Type,Null,Key,Default,Extra
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,day,bigint(2),NO,,None,
4,keyword1,bigint(100),YES,,None,
5,keyword2,bigint(100),YES,,None,
6,keyword3,bigint(100),YES,,None,
7,keyword4,bigint(100),YES,,None,
8,keyword5,bigint(100),YES,,None,


### news counting정보 주별로 저장

In [35]:
"""
db table에 저장
"""
for i in range(len(news_df)):
    query = """ 
    Insert into news_counting2 (year, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (%d, %d, %d, %d, %d, %d, %d, %d) ;
    """
    year, month, day = getDate(news_df.iloc[i]['date'])
    keyword1_cnt = news_df.iloc[i]['침체']
    keyword2_cnt = news_df.iloc[i]['금융위기']
    keyword3_cnt = news_df.iloc[i]['불황']
    keyword4_cnt = news_df.iloc[i]['저성장']
    keyword5_cnt = news_df.iloc[i]['외환위기']

    mystring = ( query % (year, month, day, keyword1_cnt, keyword2_cnt, keyword3_cnt, keyword4_cnt, keyword5_cnt) )
    print(mystring)
    cur.execute(mystring)

ear, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (2018, 5, 14, 473, 293, 162, 83, 146) ;
    
 
    Insert into news_counting2 (year, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (2018, 5, 21, 503, 212, 152, 79, 92) ;
    
 
    Insert into news_counting2 (year, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (2018, 5, 28, 633, 237, 216, 81, 89) ;
    
 
    Insert into news_counting2 (year, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (2018, 6, 4, 444, 199, 142, 99, 80) ;
    
 
    Insert into news_counting2 (year, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (2018, 6, 11, 435, 235, 133, 57, 75) ;
    
 
    Insert into news_counting2 (year, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (2018, 6, 18, 456, 229, 137, 84, 96) ;
    
 
    Insert into news_counting2 (year, month, day, keyword1, keyword2, keyword3, keyword4, keyword5 ) values (

In [36]:
# 확인
query = """ 
select * from news_counting2 ;
"""
cur.execute(query)
tmp = pd.DataFrame(list(cur.fetchall()))
tmp

,id,year,month,day,keyword1,keyword2,keyword3,keyword4,keyword5
0,1,2015,9,7,737,399,273,256,135
1,2,2015,9,14,701,513,274,232,226
2,3,2015,9,21,575,267,231,241,111
3,4,2015,9,28,480,248,173,126,79
4,5,2015,10,5,576,262,204,216,109
...,...,...,...,...,...,...,...,...,...
264,265,2020,9,28,489,90,114,27,78
265,266,2020,10,5,816,183,177,47,78
266,267,2020,10,12,857,199,176,55,91
267,268,2020,10,19,911,155,143,64,129


In [37]:
conn.commit()
cur.close()
conn.close()